## Distributed hyperparameter tuning with scikit-learn + joblib-spark

- Dataset: [Credit Card Fraud Detection, Worldline & Machine Learning Group of ULB](http://mlg.ulb.ac.be/)
  - Anonymized credit card transactions labeled as fraudulent or genuine
  - Hosted on Databricks at `/databricks-datasets/credit-card-fraud`

- Model: `sklearn.ensemble.RandomForestClassifier`
- Tuning: `sklearn.model_selection.RandomizedSearchCV`
  - scikit-learn's random search implementation with cross validation
  - Uses `joblib-spark` as a backend for distributed tuning

In [0]:
import mlflow
import pandas as pd
import scipy.stats as stats

from joblibspark import register_spark

from pyspark.ml.functions import vector_to_array

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.utils import parallel_backend

In [0]:
mlflow.sklearn.autolog(disable=True)

#### Load and prepare the data

In [0]:
spark_df = spark.read.parquet("/databricks-datasets/credit-card-fraud/data")
spark_df = spark_df.withColumn("pca", vector_to_array("pcaVector"))

pca_length = 28
pca_columns = [spark_df.pca[i] for i in range(pca_length)]

data = spark_df.select(["time", "amountRange", "label"] + pca_columns).toPandas()
data = data.fillna(data.mean())
target_col = "label"

In [0]:
data.head()

Out[34]:

,time,amountRange,label,pca[0],pca[1],pca[2],pca[3],pca[4],pca[5],pca[6],pca[7],pca[8],pca[9],pca[10],pca[11],pca[12],pca[13],pca[14],pca[15],pca[16],pca[17],pca[18],pca[19],pca[20],pca[21],pca[22],pca[23],pca[24],pca[25],pca[26],pca[27]
0,52972.0,2,0,-0.775461,0.709595,1.610122,1.231792,0.316178,0.114410,0.410964,0.048246,0.016042,0.590293,1.069637,-0.097002,-2.053954,0.296434,-0.242126,-1.103983,0.438081,-0.090098,0.972525,-0.009525,-0.031317,0.289930,-0.215881,0.196379,-0.267693,-0.298693,0.011195,-0.023320
1,41768.0,6,0,0.873554,-1.377510,1.072031,0.898335,-1.377677,1.134118,-1.049505,0.303593,0.391175,0.306457,-1.743321,0.496823,0.582594,-1.042981,-1.093048,-2.158048,0.552883,0.941660,-0.878166,-0.188112,-0.206446,-0.020860,-0.294526,-0.419986,0.530462,-0.095314,0.102470,0.061393
2,40769.0,7,0,0.890897,-0.528187,-0.678654,0.168701,-0.141127,-1.037345,0.786720,-0.463099,-0.231256,-0.210527,-0.579607,0.320749,0.739513,0.385612,0.760072,0.056286,-0.365932,-0.553207,0.225507,0.469012,0.072491,-0.283583,-0.367602,-0.003742,0.548136,1.073835,-0.156020,0.029210
3,40682.0,3,0,-0.572954,0.458246,0.239598,-1.561953,2.688063,3.529501,0.386101,0.773474,0.061670,-0.425875,-0.594523,-0.149708,-0.368630,-0.285347,-0.600322,0.106861,-0.793363,-0.208004,0.082330,0.229461,-0.290700,-0.627817,-0.206940,1.021907,0.191261,0.219703,0.122007,-0.062997
4,50032.0,7,0,-2.053059,0.504530,-0.111156,-1.040738,-2.684590,0.048439,2.152220,0.497990,-0.802817,-1.262772,1.101034,0.617550,-0.478062,0.968829,-0.337551,1.020513,-0.476524,0.048632,-0.389469,-0.142086,-0.051183,-0.370120,0.583370,0.556771,-0.250298,0.631668,0.103110,-0.186396


In [0]:
X = data.drop([target_col], axis=1)
y = data[target_col]

#### Define the search space

In [0]:
# Define the hyperparameter search space
space = {
  "n_estimators": stats.randint(50, 150),
  "criterion": ["gini", "entropy"],
  "min_samples_leaf": stats.randint(1, 20),
  "min_samples_split": stats.uniform(0, 1)
}

n_evals = 200

### Single-node hyperparameter tuning

#### Train a `RandomForestClassifier` with `RandomizedSearchCV`

In [0]:
model = RandomForestClassifier()

search = RandomizedSearchCV(
  estimator=model,
  param_distributions=space,
  n_iter=n_evals,
  n_jobs=8,
  cv=2,
  verbose=2
)
search.fit(X, y)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done 25 tasks | elapsed: 3.8min
[Parallel(n_jobs=8)]: Done 146 tasks | elapsed: 15.8min
[Parallel(n_jobs=8)]: Done 349 tasks | elapsed: 35.7min
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed: 42.4min finished
Out[37]: RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=200,
 n_jobs=8,
 param_distributions={'criterion': ['gini', 'entropy'],
 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3554892eb0>,
 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f354e2692e0>,
 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f354e269a30>},
 verbose=2)

In [0]:
best = search.best_params_
best["CV Score"] = search.best_score_
display(pd.DataFrame(best, index=[0]))

criterion,min_samples_leaf,min_samples_split,n_estimators,CV Score
entropy,15,0.0015333795911326442,148,0.9992380805616813


### Distributed hyperparameter tuning

#### Train a `RandomForestClassifier` with `RandomizedSearchCV` + `joblib-spark`

In [0]:
register_spark()

parallelism = 64
with parallel_backend("spark", n_jobs=parallelism):
  search = RandomizedSearchCV(
    estimator=model,
    param_distributions=space,
    n_iter=n_evals,
    cv=2,
    verbose=2
  )
  search.fit(X, y)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
[Parallel(n_jobs=64)]: Using backend SparkDistributedBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done 34 tasks | elapsed: 2.0min
[Parallel(n_jobs=64)]: Done 237 tasks | elapsed: 6.7min
[Parallel(n_jobs=64)]: Done 400 out of 400 | elapsed: 10.9min finished

In [0]:
best = search.best_params_
best["CV Score"] = search.best_score_
display(pd.DataFrame(best, index=[0]))

criterion,min_samples_leaf,min_samples_split,n_estimators,CV Score
gini,6,0.22455440036910468,97,0.9991678574945968
